In [ ]:
from google.colab import drive
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Function to load and preprocess images
def load_images_from_folder(folder_path, label, resize=(224, 224)):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.tif')):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert("RGB")
            img_resized = img.resize(resize)  # Resize to 224x224
            images.append(np.array(img_resized) / 255.0)  # Normalize
            labels.append(label)
    return images, labels

# Paths to original HOLES and NO HOLES datasets
holes_folder = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/HOLES'  # Replace with your path
no_holes_folder = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/NO HOLES'  # Replace with your path

# Load HOLES (defect images) and NO HOLES (non-defect images)
holes_images, holes_labels = load_images_from_folder(holes_folder, label=1)
no_holes_images, no_holes_labels = load_images_from_folder(no_holes_folder, label=0)

In [ ]:
import os

# Function to count files in a folder
def count_patches_in_folder(folder_path):
    patch_count = len([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg', '.tif'))])
    return patch_count

# Paths to the patch folders
defect_patches_folder = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/defect patches'  # Replace with your path
holes_with_patches_folder = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/PATCHES'  # Replace with your path
no_holes_with_patches_folder = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/PATCHES_nh'  # Replace with your path

# Count patches in each folder
defect_patch_count = count_patches_in_folder(defect_patches_folder)
holes_patch_count = count_patches_in_folder(holes_with_patches_folder)
no_holes_patch_count = count_patches_in_folder(no_holes_with_patches_folder)

# Print results
print(f"Number of patches in DEFECT_PATCHES: {defect_patch_count}")
print(f"Number of patches in HOLES_WITH_PATCHES: {holes_patch_count}")
print(f"Number of patches in NO_HOLES_WITH_PATCHES: {no_holes_patch_count}")


In [ ]:
# Load patch dataset
def load_patches_from_folder(folder_path, label):
    patches = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.tif')):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert("RGB")
            patches.append(np.array(img) / 255.0)  # Normalize pixel values
            labels.append(label)
    return patches, labels

# Paths to patch folders
defect_patches_folder = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/defect patches'  # Replace with your path
non_defect_patches_folder_1 = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/PATCHES'  # Replace with your path
non_defect_patches_folder_2 = '/content/drive/MyDrive/DAISY2.0 Data/DEC_data_cropped/PATCHES_nh'  # Replace with your path

# Load patches
defect_patches, defect_labels = load_patches_from_folder(defect_patches_folder, label=1)
non_defect_patches_1, non_defect_labels_1 = load_patches_from_folder(non_defect_patches_folder_1, label=0)
non_defect_patches_2, non_defect_labels_2 = load_patches_from_folder(non_defect_patches_folder_2, label=0)

# Combine all data (original + patches)
all_images = holes_images + no_holes_images + defect_patches + non_defect_patches_1 + non_defect_patches_2
all_labels = holes_labels + no_holes_labels + defect_labels + non_defect_labels_1 + non_defect_labels_2

# Convert to numpy arrays
X = np.array(all_images)
y = np.array(all_labels)

print(f"Total dataset size: {X.shape}, Labels: {y.shape}")



In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)  # 80% train, 20% temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Split temp into 10% val, 10% test

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load pre-trained VGG16
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout regularization
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train only the custom layers
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,  # Start with a few epochs
    batch_size=32,
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
# Unfreeze all layers for fine-tuning
for layer in base_model.layers:
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Learning rate scheduler and early stopping
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# Fine-tune the entire model
fine_tune_history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)


In [ ]:
# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


In [ ]:
# Accuracy plot
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(fine_tune_history.history['accuracy'], label='Fine-Tune Training Accuracy')
plt.plot(fine_tune_history.history['val_accuracy'], label='Fine-Tune Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs. Validation Accuracy')
plt.show()

# Loss plot
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.plot(fine_tune_history.history['loss'], label='Fine-Tune Training Loss')
plt.plot(fine_tune_history.history['val_loss'], label='Fine-Tune Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs. Validation Loss')
plt.show()


In [ ]:
# Predict on test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Defect', 'Defect'], yticklabels=['Non-Defect', 'Defect'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Test Set)')
plt.show()

# Classification report
print("Classification Report (Test Set):")
print(classification_report(y_test, y_pred, target_names=['Non-Defect', 'Defect']))


In [ ]:
# Predict on the training set
y_train_pred = (model.predict(X_train) > 0.5).astype("int32")

# Compute confusion matrix
cm_train = confusion_matrix(y_train, y_train_pred)

# Plot the confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Defect', 'Defect'], yticklabels=['Non-Defect', 'Defect'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Training Set)')
plt.show()

# Print classification report
print("Classification Report (Training Set):")
print(classification_report(y_train, y_train_pred, target_names=['Non-Defect', 'Defect']))


In [ ]:
# Predict on the validation set
y_val_pred = (model.predict(X_val) > 0.5).astype("int32")

# Compute confusion matrix
cm_val = confusion_matrix(y_val, y_val_pred)

# Plot the confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Defect', 'Defect'], yticklabels=['Non-Defect', 'Defect'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Validation Set)')
plt.show()

# Print classification report
print("Classification Report (Validation Set):")
print(classification_report(y_val, y_val_pred, target_names=['Non-Defect', 'Defect']))